<a href="https://colab.research.google.com/github/spatank/CIS-530/blob/master/Homework%2010/cis530_nmt_mecab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/Drive', force_remount = True)
# When you run this block, you will need to click open a link to get some 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/Drive


In [0]:
import os
# TODO
os.chdir('Drive/My Drive/CIS-530/Homework 10/data')

In [0]:
!pip install mecab-python3

In [0]:
import MeCab

wakati = MeCab.Tagger("-Owakati")

for fname in ['ja-train.txt','ja-dev.txt','ja-test.txt']:
  with open('split/' + fname, encoding = 'utf=8') as fj_orig:
    with open('mecab/mecab-' + fname, 'w', encoding = 'utf-8') as fj_tok: 
      for line in fj_orig:
        tokens = wakati.parse(line)
        fj_tok.write(tokens)

In [0]:
# Install BPE (https://github.com/rsennrich/subword-nmt)
# TODO
!pip install subword-nmt

In [0]:
# Install OpenNMT (https://github.com/OpenNMT/OpenNMT-py)
# TODO
!pip install OpenNMT-py

In [0]:
# Apply BPE, following instructions from the Github repo
# this takes around 15 minutes

# TODO
# codes_file is where the BPE information is stored. Name it as you like. 

# Learn and apply BPE for English data

!subword-nmt learn-bpe -s 10000 < 'mecab/mecab-ja-train.txt' > 'mecab/mecab-ja-bpe' # learn from train
!subword-nmt apply-bpe -c 'mecab/mecab-ja-bpe' < 'mecab/mecab-ja-train.txt' > 'mecab/mecab-ja-train-bpe-out.txt' # apply to train
!subword-nmt apply-bpe -c 'mecab/mecab-ja-bpe' < 'mecab/mecab-ja-dev.txt' > 'mecab/mecab-ja-dev-bpe-out.txt' # apply to dev
!subword-nmt apply-bpe -c 'mecab/mecab-ja-bpe' < 'mecab/mecab-ja-test.txt' > 'mecab/mecab-ja-test-bpe-out.txt' # apply to test

In [0]:
# Run OpenNMT, following instructions from the Github repo
# this takes 1-2 hours

# TODO
# Name the prefixes for data and checkpoints as you like
!onmt_preprocess -train_src 'mecab/mecab-ja-train-bpe-out.txt' -train_tgt 'bpe/en-train-bpe-out.txt' -valid_src 'mecab/mecab-ja-dev-bpe-out.txt' -valid_tgt 'bpe/en-dev-bpe-out.txt' -save_data 'mecab/demo' -overwrite

In [0]:
!onmt_train -gpu_ranks 0 -data 'mecab/demo' -save_model demo-model

In [0]:
!onmt_translate -model 'mecab/demo-model_step_100000.pt' -src 'mecab/mecab-ja-test.txt' -output 'mecab/pred-mecab.txt' -replace_unk -verbose

In [0]:
# reverse BPE
!sed -r 's/(@@ )|(@@ ?$)//g' 'mecab/pred-mecab.txt' > 'mecab/pred.txt'

In [0]:
# Evaluate translations using BLEU
!wget https://raw.githubusercontent.com/moses-smt/mosesdecoder/master/scripts/generic/multi-bleu.perl
# TODO
!perl multi-bleu.perl 'split/en-test.txt' < 'mecab/pred.txt'

In [0]:
import subprocess

with open('split/en-test.txt', encoding = 'utf-8') as ref_file:
  for idx, line in enumerate(ref_file):
    if idx < 20:
      line_file = 'ref_line_%d.txt' % idx
      with open('mecab/lines/ref_lines/' + line_file, 'w', encoding = 'utf-8') as lf:
        lf.write(line)

with open('mecab/pred.txt', encoding = 'utf-8') as pred_file:
  for idx, line in enumerate(pred_file):
    if idx < 20:
      line_file = 'pred_line_%d.txt' % idx
      with open('mecab/lines/pred_lines/' + line_file, 'w', encoding = 'utf-8') as lf:
        lf.write(line)

for idx in range(20):
  ref_file = 'mecab/lines/ref_lines/ref_line_%d.txt' % idx
  pred_file = 'mecab/lines/pred_lines/pred_line_%d.txt' % idx
  with open(ref_file, encoding = 'utf-8') as ref:
    for line in ref:
      print(idx, line)
  with open(pred_file, encoding = 'utf-8') as pred:
    for line in pred:
      print(idx, line)
  cp = subprocess.run("perl multi-bleu.perl '%s' < '%s'" % (ref_file, pred_file), universal_newlines = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE, shell = True)
  print(cp.stdout)

In [0]:
bleu_indiv = [0, 100, 0, 43.47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
bleu = 5.84
# Adequacy: how much of the meaning in the gold standard is present in the translation?
adequacy = [0, 3, 2, 1, 1, 2, 0, 3, 2, 3, 0, 0, 1, 1, 1, 0, 1, 3, 1, 0]
# Fluency: to what extent is the translation error free, grammatically as well as spelling wise?
fluency = [3, 3, 3, 0, 1, 2, 2, 3, 3, 3, 2, 1, 3, 2, 3, 1, 3, 3, 3, 1]

In [0]:
!pip install rouge
from rouge import FilesRouge

files_rouge = FilesRouge()

hyp_path = 'mecab/pred.txt' 
ref_path = 'split/en-test.txt'
scores_indiv = files_rouge.get_scores(hyp_path, ref_path)
scores_avg = files_rouge.get_scores(hyp_path, ref_path, avg = True)

In [0]:
for idx in range(20):
  print(scores_indiv[:20][idx]['rouge-l']['f'])

In [0]:
rouge_indiv = [0, 1, 0.66, 0.77, 0.43, 0.22, 0.21, 0.50, 0.35, 0.40, 0, 0.13, 0, 0.46, 0, 0, 0.14, 0.29, 0.11, 0.40]
rouge = 0.27 # from scores_avg

In [0]:
meteor_indiv = [0, 1, 0.17, 0.64, 0.11, 0.08, 0.08, 0.25, 0.34, 0.11, 0, 0.07, 0, 0.13, 0, 0, 0.04, 0.06, 0.03, 0.09]
meteor = 0.15

In [0]:
from scipy import stats

print(stats.spearmanr(bleu_indiv, adequacy))
print(stats.spearmanr(bleu_indiv, fluency))

print('\n')

print(stats.spearmanr(rouge_indiv, adequacy))
print(stats.spearmanr(rouge_indiv, fluency))

print('\n')

print(stats.spearmanr(meteor_indiv, adequacy))
print(stats.spearmanr(meteor_indiv, fluency))

SpearmanrResult(correlation=0.2383458646616541, pvalue=0.3115560243964671)
SpearmanrResult(correlation=-0.13709815216348134, pvalue=0.5643611664547012)


SpearmanrResult(correlation=0.5804294924639211, pvalue=0.007292772743603303)
SpearmanrResult(correlation=-0.08162468972950063, pvalue=0.7322727962473675)


SpearmanrResult(correlation=0.5553712690247996, pvalue=0.011018695268952019)
SpearmanrResult(correlation=-0.07495591574420851, pvalue=0.7534671680911954)


In [0]:
# Run NMT system on jp_test_sentences.txt 

wakati = MeCab.Tagger("-Owakati")

with open('jp_test_sentences.txt', encoding = 'utf-8') as test_file:
  with open('jp-test-sentences-mecab.txt', 'w', encoding = 'utf-8') as mecab_test_file:
    for line in test_file:
      tokens = wakati.parse(line)
      mecab_test_file.write(tokens)

# BPE for jp-test-sentences-mecab.txt
!subword-nmt apply-bpe -c 'mecab/mecab-ja_bpe' < 'jp-test-sentences-mecab.txt' > 'jp-test-sentences-mecab-bpe-out.txt' # apply to test
!onmt_translate -model 'mecab/demo-model_step_100000.pt' -src 'jp-test-sentences-mecab-bpe-out.txt' -output 'translations-mecab-bpe.txt' -replace_unk -verbose
!sed -r 's/(@@ )|(@@ ?$)//g' 'translations-mecab-bpe.txt' > 'translations.txt'